In [1]:
import os
import tempfile
from pathlib import Path
import pandas as pd
import numpy as np
import pyBigWig
import importlib
import xarray as xr
import tqdm
import pandas as pd
import crandata
crandata = importlib.reload(crandata)


import crandata
import crandata.chrom_io
crandata.crandata = importlib.reload(crandata.crandata)
crandata.chrom_io = importlib.reload(crandata.chrom_io)
crandata._anndatamodule = importlib.reload(crandata._anndatamodule)
# crandata._dataloader = importlib.reload(crandata._dataloader)
# crandata._dataset = importlib.reload(crandata._dataset)
crandata._anndatamodule = importlib.reload(crandata._anndatamodule)
from crandata._anndatamodule import MetaAnnDataModule

# Create temporary directories for beds, bigwigs, etc.
temp_dir = tempfile.TemporaryDirectory()
base_dir = Path(temp_dir.name)
beds_dir = base_dir / "beds"
bigwigs_dir = base_dir / "bigwigs"
beds_dir.mkdir(exist_ok=True)
bigwigs_dir.mkdir(exist_ok=True)

# Create a chromsizes file
chromsizes_file = base_dir / "chrom.sizes"
with open(chromsizes_file, "w") as f:
    f.write("chr1\t1000\n")

# Create two BED files (ClassA and ClassB)
bed_data_A = pd.DataFrame({0: ["chr1", "chr1"],
                           1: [100, 300],
                           2: [200, 400]})
bed_data_B = pd.DataFrame({0: ["chr1", "chr1"],
                           1: [150, 350],
                           2: [250, 450]})
bed_file_A = beds_dir / "ClassA.bed"
bed_file_B = beds_dir / "ClassB.bed"
bed_data_A.to_csv(bed_file_A, sep="\t", header=False, index=False)
bed_data_B.to_csv(bed_file_B, sep="\t", header=False, index=False)

# Create a consensus BED file
consensus = pd.DataFrame({0: ["chr1", "chr1", "chr1"],
                          1: [100, 300, 350],
                          2: [200, 400, 450]})
consensus_file = base_dir / "consensus.bed"
consensus.to_csv(consensus_file, sep="\t", header=False, index=False)

# Create a bigWig file with a single chromosome region
bigwig_file = bigwigs_dir / "test.bw"
bw = pyBigWig.open(str(bigwig_file), "w")
bw.addHeader([("chr1", 1000)])
bw.addEntries(chroms=["chr1"], starts=[0], ends=[1000], values=[5.0])
bw.close()

# Set parameters for extraction
target_region_width = 100
backed_path = os.path.join(base_dir, "chrom_data.h5")

# Create the CrAnData object from the bigWig files and consensus regions
adata = crandata.chrom_io.import_bigwigs(
    bigwigs_folder=bigwigs_dir,
    regions_file=consensus_file,
    backed_path=backed_path,
    target_region_width=target_region_width,
    chromsizes_file=chromsizes_file,
    
)

# Add a random obsm entry
adata.obsm['gex'] = xr.DataArray(np.random.randn(adata.obs.shape[0], 100),
                                 dims=['types', 'genes'])

# Create a synthetic BEDP file for Hi-C contacts and add contacts to adata.varp
synthetic_bedp = pd.DataFrame({
    0: ["chr1", "chr1"],
    1: [100, 300],
    2: [200, 400],
    3: ["chr1", "chr1"],
    4: [150, 350],
    5: [250, 450],
    6: [10, 20]
})
synthetic_bedp_file = base_dir / "synthetic.bedp"
synthetic_bedp.to_csv(synthetic_bedp_file, sep="\t", header=False, index=False)

from crandata.chrom_io import add_contact_strengths_to_varp
contacts = add_contact_strengths_to_varp(adata, [str(synthetic_bedp_file)], key="hic_contacts")

print("Added Hi-C contact data to adata.varp['hic_contacts']:")
print("Shape:", adata.varp["hic_contacts"].shape)
print(adata.varp["hic_contacts"])


# Write to HDF5 and load back.
h5_path = os.path.join(base_dir, "adata.h5")
adata.to_h5(h5_path)
adata_loaded = crandata.crandata.CrAnData.from_h5(h5_path,backed=['X'])
print("\nDirectory contents:", os.listdir(base_dir))
print("\nLoaded CrAnData from HDF5:")
print(adata_loaded)
print("obs:")
print(adata_loaded.obs)
print("var:")
print(adata_loaded.var)
print("varp keys:", list(adata_loaded.varp.keys()))
if "hic_contacts" in adata_loaded.varp:
    print("Hi-C contact data shape:", adata_loaded.varp["hic_contacts"].shape)
    print(adata_loaded.varp["hic_contacts"])


# Create two copies of the loaded CrAnData (simulate two different datasets/species)
adata1 = adata_loaded.copy()
adata2 = adata_loaded.copy()
# Ensure each has a 'split' column
adata1.var["split"] = "train"
adata2.var["split"] = "train"

# Create a dummy FASTA file for the genome (with a single record for chr1)
fasta_file = base_dir / "chr1.fa"
with open(fasta_file, "w") as f:
    f.write(">chr1\n")
    f.write("A" * 1000 + "\n")

# Instead of passing a string, create a Genome object.
from crandata._genome import Genome
dummy_genome = Genome(str(fasta_file), chrom_sizes=str(chromsizes_file))

# Import MetaAnnDataModule (using the package name so that relative imports resolve)


# Instantiate MetaAnnDataModule with the two datasets and corresponding genomes.
meta_module = MetaAnnDataModule(
    adatas=[adata1, adata2],
    genomes=[dummy_genome, dummy_genome],
    data_sources={'y': 'X','hic':'varp/hic_contacts','gex':'obsm/gex'},
    in_memory=True,
    random_reverse_complement=True,
    max_stochastic_shift=5,
    deterministic_shift=False,
    shuffle_obs=True,
    shuffle=True,
    batch_size=3,    # small batch size for testing
    epoch_size=10    # small epoch size for quick testing
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("fit")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm.tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    # if i == 1:
    #     break

print(os.listdir(base_dir))

temp_dir.cleanup()


100%|██████████| 1/1 [00:00<00:00, 7681.88it/s]
2025-03-10 13:57:50.674 | INFO     | crandata.chrom_io:import_bigwigs:417 - Extracting values from 1 bigWig files...
1it [00:00, 93.27it/s]


Added Hi-C contact data to adata.varp['hic_contacts']:
Shape: (3, 3, 1)
<xarray.DataArray (var_0: 3, var_1: 3, hic_file: 1)> Size: 140B
<COO: shape=(3, 3, 1), dtype=float32, nnz=5, fill_value=0.0>
Coordinates:
  * var_0     (var_0) object 24B 'chr1:100-200' 'chr1:300-400' 'chr1:350-450'
  * var_1     (var_1) object 24B 'chr1:100-200' 'chr1:300-400' 'chr1:350-450'
  * hic_file  (hic_file) int64 8B 0

Directory contents: ['beds', 'bigwigs', 'chrom.sizes', 'consensus.bed', 'chrom_data.h5', 'synthetic.bedp', 'adata.h5']

Loaded CrAnData from HDF5:
CrAnData object with 1 observations and 3 variables
obs:
                                            file_path
test  /scratch/fast/46144/tmpjwrhoucu/bigwigs/test.bw
var:
             chrom  start  end  chunk_index   chr
region                                           
chr1:100-200  chr1    100  200            0  chr1
chr1:300-400  chr1    300  400            0  chr1
chr1:350-450  chr1    350  450            0  chr1
varp keys: ['hic_contacts']
Hi

100%|██████████| 3/3 [00:00<00:00, 12774.53it/s]

2025-03-10T13:58:02.434586-0700 INFO Loading sequences into memory...



100%|██████████| 3/3 [00:00<00:00, 16426.78it/s]

...

2025-03-10T13:58:02.458052-0700 INFO Loading sequences into memory...



100%|██████████| 3/3 [00:00<00:00, 4529.49it/s]
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/torch/utils/data/sampler.py:76: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn(



Iterating over a couple of training batches from MetaAnnDataModule:


100%|██████████| 4/4 [00:01<00:00,  2.04it/s]

Meta Batch 0:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 1:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 2:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 3:
  sequence: shape torch.Size([100, 1, 4])
  y: shape torch.Size([1, 1, 100])
  hic: shape torch.Size([1, 1, 1])
  gex: shape torch.Size([1, 1, 100])
['beds', 'bigwigs', 'chrom.sizes', 'consensus.bed', 'chrom_data.h5', 'synthetic.bedp', 'adata.h5', 'chr1.fa', 'chr1.fa.fai']


In [2]:
import cProfile

code = '''
for i, batch in enumerate(meta_train_dl.data):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
'''

cProfile.run(code)


Meta Batch 0:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 1:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 2:
  sequence: shape torch.Size([100, 3, 4])
  y: shape torch.Size([1, 3, 100])
  hic: shape torch.Size([1, 3, 1])
  gex: shape torch.Size([1, 3, 100])
Meta Batch 3:
  sequence: shape torch.Size([100, 1, 4])
  y: shape torch.Size([1, 1, 100])
  hic: shape torch.Size([1, 1, 1])
  gex: shape torch.Size([1, 1, 100])
         19904 function calls (19690 primitive calls) in 0.024 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1128(find_spec)
       60    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1222(_

In [3]:
adata1.varp['hic_contacts'].data.todense()

array([[[10.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [20.],
        [20.]],

       [[ 0.],
        [20.],
        [20.]]], dtype=float32)

In [4]:
dir(adata1.varp['hic_contacts'].data)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_function__',
 '__array_namespace__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__complex__',
 '__dask_tokenize__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__metaclass__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',


In [5]:
dfssfd

NameError: name 'dfssfd' is not defined

In [ ]:
import crandata
import importlib
crandata = importlib.reload(crandata)
crandata.chrom_io = importlib.reload(crandata.chrom_io)
crandata.crandata = importlib.reload(crandata.crandata)


In [ ]:
import crandata
import os

genome_path = '/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/genome/onehots/mouse'
fasta_file = os.path.join(genome_path,'mouse.fa')
chrom_sizes = os.path.join(genome_path,'mouse.fa.sizes')
annotation_gtf_file = os.path.join(genome_path,'mouse.annotation.gtf')

genome = crandata.Genome(fasta_file, chrom_sizes, annotation_gtf_file)

# Set parameters for binning.
WINDOW_SIZE = 2114
OFFSET = WINDOW_SIZE // 2  # e.g., 50% overlap
N_THRESHOLD = 0.3

# Optionally specify an output path for the BED file.
OUTPUT_BED = os.path.join(genome_path, "binned_genome.bed")

# Generate bins and optionally write to disk.
binned_df = crandata.bin_genome(genome, WINDOW_SIZE, OFFSET, n_threshold=N_THRESHOLD, output_path=OUTPUT_BED)

print("Filtered bins:")
print(binned_df)


In [ ]:

bigwigs_dir = '/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/SpinalCord/manuscript/ATAC/mouse/Group_bigwig/'
n_bins = WINDOW_SIZE//50

adata = crandata.chrom_io.import_bigwigs(
    bigwigs_folder=bigwigs_dir,
    regions_file=OUTPUT_BED,
    backed_path='/home/matthew.schmitz/Matthew/mouse_spc_test.h5',
    target_region_width=WINDOW_SIZE,
    chromsizes_file=chrom_sizes,
    n_bins=n_bins   
)

In [ ]:
adata

In [ ]:
adata.X[0,:,:]

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
adata = crandata.crandata.CrAnData.from_h5('/home/matthew.schmitz/Matthew/mouse_spc_test.h5')

In [ ]:
import crandata
import os
import crested
from tqdm import tqdm

In [ ]:
genomes = {}
beds = {}
chromsizes_files = {}
bed_files = {}
species = ['mouse','human','macaque']

WINDOW_SIZE = 2114
OFFSET = WINDOW_SIZE // 2  # e.g., 50% overlap
N_THRESHOLD = 0.3
n_bins = WINDOW_SIZE//50

In [ ]:
for s in species:
    genome_path = '/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/genome/onehots/'+s
    fasta_file = os.path.join(genome_path,s+'.fa')
    chrom_sizes = os.path.join(genome_path,s+'.fa.sizes')
    annotation_gtf_file = os.path.join(genome_path,s+'.annotation.gtf')
    chromsizes_files[s] = chrom_sizes
    genome = crandata.Genome(fasta_file, chrom_sizes, annotation_gtf_file)
    genomes[s] = genome
    # Set parameters for binning.
    
    # Optionally specify an output path for the BED file.
    OUTPUT_BED = os.path.join(genome_path, "binned_genome.bed")
    bed_files[s] = OUTPUT_BED
    # Generate bins and optionally write to disk.
    # binned_df = crandata.bin_genome(genome, WINDOW_SIZE, OFFSET, n_threshold=N_THRESHOLD, output_path=OUTPUT_BED)
    # print("Filtered bins:")
    # print(binned_df)

In [ ]:
adatas = {}

for s in species:
    bigwigs_dir = os.path.join('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/SpinalCord/manuscript/ATAC',s,'Group_bigwig')
    # adata = crandata.chrom_io.import_bigwigs(
    #     bigwigs_folder=bigwigs_dir,
    #     regions_file=bed_files[s],
    #     backed_path='/home/matthew.schmitz/Matthew/'+s+'_spc_test.h5',
    #     target_region_width=WINDOW_SIZE,
    #     chromsizes_file=chromsizes_files[s],
    #     n_bins=n_bins
    # )
    # adatas[s] = adata
    adatas[s] = crandata.crandata.CrAnData.from_h5('/home/matthew.schmitz/Matthew/'+s+'_spc_test.h5')
    

In [ ]:
import numpy as np
adatas['mouse'].uns['chunk_size'] = 512
adatas['human'].uns['chunk_size'] = 512
adatas['macaque'].uns['chunk_size'] = 512
adatas['mouse'].var["chunk_index"] = np.arange(adatas['mouse'].var.shape[0]) // 512
adatas['human'].var["chunk_index"] = np.arange(adatas['human'].var.shape[0]) // 512
adatas['macaque'].var["chunk_index"] = np.arange(adatas['macaque'].var.shape[0]) // 512


In [ ]:
for s in adatas.keys():
    crested.pp.train_val_test_split(
        adatas[s], strategy="region", val_size=0.1, test_size=0.1, random_state=42
    )


In [ ]:
import crandata
import importlib
importlib.reload(crandata)
importlib.reload(crandata.crandata)
importlib.reload(crandata._anndatamodule)
importlib.reload(crandata._dataloader)
importlib.reload(crandata._dataset)

In [ ]:
meta_module = crandata._anndatamodule.MetaAnnDataModule(
    adatas=list(adatas.values()),
    genomes=list(genomes.values()),
    data_sources={'y': 'X'},
    in_memory=True,
    random_reverse_complement=True,
    max_stochastic_shift=10,
    deterministic_shift=False,
    shuffle_obs=False,
    shuffle=True,
    batch_size=32,    # small batch size for testing
    epoch_size=1000000    # small epoch size for quick testing
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("fit")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break


In [ ]:
import cProfile

code = '''
for i, batch in enumerate(meta_train_dl.data):
    # print(f"Meta Batch {i}:")
    # for key, tensor in batch.items():
    #     print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break
'''

out = cProfile.run(code,sort=True)


In [ ]:
model_architecture = crested.tl.zoo.chrombpnet(
    seq_len=2114, num_classes=51
)


In [ ]:
import keras
# Create your own configuration
# I recommend trying this for peak regression with a weighted cosine mse log loss function
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss = crested.tl.losses.CosineMSELogLoss(max_weight=100, multiplier=1)
metrics = [
    keras.metrics.MeanAbsoluteError(),
    keras.metrics.MeanSquaredError(),
    keras.metrics.CosineSimilarity(axis=1),
    crested.tl.metrics.PearsonCorrelation(),
    crested.tl.metrics.ConcordanceCorrelationCoefficient(),
    crested.tl.metrics.PearsonCorrelationLog(),
    crested.tl.metrics.ZeroPenaltyMetric(),
]

alternative_config = crested.tl.TaskConfig(optimizer, loss, metrics)
print(alternative_config)


In [ ]:
trainer = crested.tl.Crested(
    data=datamodule,
    model=model_architecture,
    config=alternative_config,
    project_name="mouse_biccn",  # change to your liking
    run_name="basemodel",  # change to your liking
    logger="wandb",  # or None, 'dvc', 'tensorboard'
    seed=7,  # For reproducibility
)
# train the model
trainer.fit(
    epochs=60,
    learning_rate_reduce_patience=3,
    early_stopping_patience=6,
)
